In [2]:
import os
import re
import pandas as pd
from collections import defaultdict
from dataclasses import dataclass


In [3]:
@dataclass
class ChannelStat:
    utilized_cycles: int = 0
    transaction_count: int = 0

import re
import pandas as pd
from dataclasses import dataclass

@dataclass
class ChannelStat:
    utilized_cycles: int = 0
    transaction_count: int = 0
    is_active: bool = False
    active_start_cycle: int = 0

BURST_CYCLES = 2

def analyze_channel_utilization(log_filename="log.txt", num_channels=16, verbose=True):
    stats = [ChannelStat() for _ in range(num_channels)]

    cycle_pattern = re.compile(r"\[\+\]\s*(\d+)")
    cmd_pattern = re.compile(r"\(Issue\)\s+Type:\s+(READ|WRITE)\s+\|\s+Addr:\s+0x[0-9a-fA-F]+ -> Channel:\s+(-?\d+),")

    current_cycle = 0

    with open(log_filename, "r") as f:
        for line in f:
            if cycle_match := cycle_pattern.search(line):
                current_cycle = int(cycle_match.group(1))

            if cmd_match := cmd_pattern.search(line):
                cmd_type, ch_str = cmd_match.groups()
                ch = int(ch_str)
                if 0 <= ch < num_channels:
                    stats[ch].transaction_count += 1

    # Calculate burst-based utilized_cycles
    for stat in stats:
        stat.utilized_cycles = stat.transaction_count * BURST_CYCLES

    return stats, current_cycle


def print_channel_stats(stats, total_cycles):
    print(f"Cycles : {total_cycles}")
    for ch, stat in enumerate(stats):
        utilization = stat.utilized_cycles / total_cycles * 100 if total_cycles > 0 else 0
        print(f"Ch{ch:2d} : {utilization:6.2f}% ({stat.transaction_count}, {stat.utilized_cycles})")



In [53]:
stats, total_cycles = analyze_channel_utilization("log.txt")
print_channel_stats(stats, total_cycles)

Cycles : 375
Ch 0 :  17.07% (32, 64)
Ch 1 :  17.07% (32, 64)
Ch 2 :  17.07% (32, 64)
Ch 3 :  17.07% (32, 64)
Ch 4 :  17.07% (32, 64)
Ch 5 :  17.07% (32, 64)
Ch 6 :  17.07% (32, 64)
Ch 7 :  17.07% (32, 64)
Ch 8 :  17.07% (32, 64)
Ch 9 :  17.07% (32, 64)
Ch10 :  17.07% (32, 64)
Ch11 :  17.07% (32, 64)
Ch12 :  17.07% (32, 64)
Ch13 :  17.07% (32, 64)
Ch14 :  17.07% (32, 64)
Ch15 :  17.07% (32, 64)
